# Parameter Senstivity and Monte Carlo Simulation 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

#Working with masked array
import numpy.ma as ma
import math

# One at time Parameter Sensitivity 

In [ ]:
#Pgrad, Tgrad, Area, CFR, CFRO, LA= params2
free_params= [0.65,0.5, 25, 0.01, 0.1, 0.138]

param_names = ["Pcorr", "Scorr", "Tx", "Ts", "CX", "FC", "Beta", "LP",
               "K1", "K2", "UZ1", "KLZ", "PERC"]

initial_params = [1, 1, -0.36, 1.15, 2.654, 61.8, 4.5, 0.8, 0.022, 0.06, 82, 0.007, 0.65]
bounds =[(1,1), (1,1), (-1.0,1.0), (-0.5,1.5), (1.5,3.5), (50,100), (1,6),
        (0.75, 0.85), (0.01, 0.045), (0.045, 0.1), (40,100), (0.003, 0.01), (0.1,1)]

catchments = [
    ('subbasin1', 'subbasin1','subbasin1'),
    ('subbasin2','subbasin2','subbasin2'),
    ('subbasin3',  'subbasin3','subbasin3'),
    ]

n_sweep = 30  # number of parameter values per sweep

sweep_rows = []

for i_param, (pname, (low, high)) in enumerate(zip(param_names, bounds)):
    sweep_values = np.linspace(low, high, n_sweep)
    for sweep_idx, pval in enumerate(sweep_values):
        params = initial_params.copy()
        params[i_param] = pval

        out_row = {
            "sweep_param": pname,
            "sweep_value": pval,
            "sweep_idx": sweep_idx,
        }
        # Store parameter values 
        out_row.update({p: params[j] for j, p in enumerate(param_names)})

        try:
            df_simq_oat = hbv_model_maxbas(
                params=params,
                params2=free_params,
                prec=prec,
                temp=temp,
                evap=evap,
                ds_oul=ds_oul
            )
            for name, obs_col, sim_col in catchments:
                obs_cal = df_qobs[obs_col].loc["1990":"2010"]
                sim_cal = df_simq_oat[sim_col].loc["1990":"2010"]
                out_row[f"NSE_{name}"] = NSE(obs_cal, sim_cal)
            out_row["status"] = "ok"
        except Exception as e:
            out_row["status"] = "fail"
            out_row["error"] = str(e)[:500]
            for name, _, _ in catchments:
                out_row[f"NSE_{name}"] = np.nan

        sweep_rows.append(out_row)

# To DataFrame
df_oat_nse = pd.DataFrame(sweep_rows)
df_oat_nse


## Monte Carlo Simulation

In [ ]:
###MC with MaxBAS
param_names = ["Pcorr", "Scorr", "Tx", "Ts", "CX", "FC", "Beta", "LP",
               "K1", "K2", "UZ1", "KLZ", "PERC"]

initial_params = [1, 1, -0.36, 1.15, 2.654, 61.8, 4.5, 0.8, 0.022, 0.06, 82, 0.007, 0.65]
bounds =[(1,1), (1,1), (-1.0,1.0), (-0.5,1.5), (1.5,3.5), (50,100), (1,6),
        (0.75, 0.85), (0.01, 0.045), (0.045, 0.1), (40,100), (0.003, 0.01), (0.1,1)]



catchments = [
    ('subbasin1', 'subbasin1','subbasin1'),
    ('subbasin2','subbasin2','subbasin2'),
    ('subbasin3',  'subbasin3','subbasin3'),


# -----------------------------
# 1) Monte Carlo sampling
# -----------------------------
def monte_carlo_params(n_sets=1000, bounds=bounds, names=param_names, seed=42):
    rng = np.random.default_rng(seed)
    low = np.array([b[0] for b in bounds], dtype=float)
    high = np.array([b[1] for b in bounds], dtype=float)
    X = rng.uniform(low=low, high=high, size=(n_sets, len(bounds)))
    return pd.DataFrame(X, columns=names)

# Generate MC parameter sets 
df_mc = monte_carlo_params(n_sets=1000, seed=123)
df_init = pd.DataFrame([initial_params], columns=param_names)
df_params_all = pd.concat([df_init, df_mc], ignore_index=True)  # run 0 = initial


# 2) Run HBV + compute NSEs 
rows = []

for run_id, row in df_params_all.iterrows():
    params = row[param_names].to_numpy(dtype=float)

    out_row = {"run": run_id}
    # store parameters
    out_row.update({p: row[p] for p in param_names})

    try:
        df_simq_r = hbv_model_maxbas(
            params=params,
            params2=free_params,
            prec=prec,
            temp=temp,
            evap=evap,
            ds_oul=ds_oul
        )

        # compute NSE per catchment (calibration period)
        for name, obs_col, sim_col in catchments:
            
            obs_cal = df_qobs[obs_col].loc["2011":"2020"]
            sim_cal = df_simq_r[sim_col].loc["2011":"2020"]
            out_row[f"NSE_{name}"] = NSE(obs_cal, sim_cal)

        #out_row["status"] = "ok"

    except Exception as e:
        out_row["status"] = "failed"
        out_row["error"] = str(e)[:1000]
        
        for name, _, _ in catchments:
            out_row[f"NSE_{name}"] = np.nan

    rows.append(out_row)

df_params_perf = pd.DataFrame(rows)
